In [69]:
# bước 1
import web3
import solcx
import json

In [70]:
print("Web3",web3.__version__)

Web3 7.10.0


In [71]:
# bước 2
from solcx import install_solc,compile_standard

install_solc('0.8.0')
print('phiên bản đã cài',solcx.get_installed_solc_versions())

phiên bản đã cài [<Version('0.8.0')>]


In [72]:
# bước 3
with open ("Message.sol","r") as file:
    source_code=file.read()
    
# kiểm tra đã đọc đúng chưa
print(source_code[:200])

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract MessageContract{
    string public message;

    constructor(string memory _message){
        message = message;
    }

    function u


# biên dịch hợp đồng comile_standerd

In [73]:
compiled_sol=compile_standard({
    "language":"Solidity",
    "sources":{
        "Message.sol":{
            "content":source_code
        }
    },
    "settings":{
        "outputSelection":{
            "*":{
                "*":["abi","metadata","evm.bytecode","evm.sourceMap"]
            }
        }
    }
},solc_version="0.8.0")

In [74]:
# 3.3 
abi=compiled_sol["contracts"]['Message.sol']['MessageContract']['abi']
bytecode=compiled_sol['contracts']['Message.sol']['MessageContract']['evm']['bytecode']['object']
print("abi rút gọn:",abi[:1])
print("\n bytecode rút gọn:",bytecode[:10],"...")

abi rút gọn: [{'inputs': [{'internalType': 'string', 'name': '_message', 'type': 'string'}], 'stateMutability': 'nonpayable', 'type': 'constructor'}]

 bytecode rút gọn: 6080604052 ...


In [75]:
from web3 import Web3  # đảm bảo bạn đã import đúng

# Kết nối tới Ganache
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))

# Kiểm tra kết nối
print("Kết nối thành công:", w3.is_connected())

# Lấy danh sách tài khoản
accounts = w3.eth.accounts
w3.eth.default_account = accounts[0]
print("Tài khoản mặc định:", accounts[:1])


Kết nối thành công: True
Tài khoản mặc định: ['0x03568dB3389c611073357F00c281209d5f302B0c']


In [76]:
# Tạo đối tượng contract từ ABI và Bytecode
MessageContract = w3.eth.contract(abi=abi, bytecode=bytecode)

# Gửi giao dịch triển khai contract, truyền constructor message
tx_hash = MessageContract.constructor("Chào Quý khách!").transact()

# Chờ nhận kết quả giao dịch để lấy địa chỉ contract
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

# Lấy địa chỉ contract vừa triển khai
contract_address = tx_receipt['contractAddress']
print("Hợp đồng đã triển khai tại địa chỉ:", contract_address)



Hợp đồng đã triển khai tại địa chỉ: 0xD754501829EA521920cEaFF40961d4BaD02cF7A0


In [77]:
print("Contract Address:",contract_address)

Contract Address: 0xD754501829EA521920cEaFF40961d4BaD02cF7A0


In [79]:
# Bước 5: Tương tác với smart contract
# Tạo instance của contract tại địa chỉ đã triển khai
message_contract_instance = w3.eth.contract(
    address=contract_address,
    abi=abi
)
# Đọc message hiện tại
current_message = message_contract_instance.functions.message().call()
print("Nội dung hiện tại:", current_message)

# Gửi giao dịch để cập nhật nội dung
tx_hash = message_contract_instance.functions.updateMessage("Xin chào Blockchain!").transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

# Đọc lại message sau cập nhật
updated_message = message_contract_instance.functions.message().call()
print("Nội dung sau cập nhật:", updated_message)


Nội dung hiện tại: 
Nội dung sau cập nhật: Xin chào Blockchain!
